In [1]:
#----------------------------------------
#libraries that need to be installed extra , if already do not exists
#mglearn, pandas, numpy, sklearn
#----------------------------------------
#---------------------------------------------------------------------------
#FLAG used in each code block mean 
#                1: that can be executed at any machine with final csv file
#                0: that cannot be executed in any machine, excpet Alex.
#--------------------------------------------------------------------------

#import libaries

import matplotlib.pyplot as plt 
import mglearn, csv, random , matplotlib
import pandas as pd
import numpy as np 
import seaborn as sns



from sklearn.feature_selection import SelectKBest, chi2
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.feature_selection import SelectFromModel

#classification models
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression, Lasso
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.mixture import GaussianMixture

#clustering
from sklearn.cluster import MeanShift, estimate_bandwidth, DBSCAN
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
import umap.umap_ as umap
from scipy.cluster.hierarchy import linkage, dendrogram

#data scaling 
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler
from collections import defaultdict

#accuracy score metric
from sklearn.metrics import accuracy_score


#cosine similatiry metric
from sklearn.metrics.pairwise import cosine_similarity

#covariance with ellipticEnvelope (anomaly detection)
from sklearn.covariance import EllipticEnvelope

#word2vec
from gensim.models import Word2Vec

#oversampling 
from imblearn.combine import SMOTETomek

#jupiter display dataset without limits 
from IPython.display import display
pd.options.display.max_columns = None

## Read data from csv file with features and merge it with graph features and labels from label file

### Use of mix of bot labels from Botsentiel and Botometer
### !!!! 
### Important combine_data are used only on Alex machine since i have all files that is combined to final csv file with all features no need to run 
### !!!!

In [2]:
#---------------------------------------------------------------------------
#FLAG  -> 0: that cannot be executed in any machine, excpet Alex.
#--------------------------------------------------------------------------

def word_2_vec(dataFrame):
    model = Word2Vec.load("/Users/shevtsov/USBotDetection/data/word2vec_1.model")
    temp = [] 
    for word in dataFrame["mst_fr_hs_tw_word"]:
        if word[3:] == "#_":
            score = [0]*10
        else:
            score = model.wv[word[3:]]
        temp.append(list(score))

        #for i in range(len(score)):
        #    temp[i].append(score[i])
    col = ["hs_tw_v_{}".format(i) for i in range(10)]
    hs_1 = pd.DataFrame(temp, columns=col)
    temp = [] 
    for word in dataFrame["mst_fr_hs_rt_word"]:
        if word[3:] == "#_":
            score = [0]*10
        else:
            score = model.wv[word[3:]]
        temp.append(list(score))

        #for i in range(len(score)):
        #    temp[i].append(score[i])
    col = ["hs_rt_v_{}".format(i) for i in range(10)]
    hs_2 = pd.DataFrame(temp, columns=col)
    
    temp = [] 
    for word in dataFrame["mst_fr_ment_tw_word"]:
        if word[3:] == "@_":
            score = [0]*10
        else:
            score = model.wv[word[3:]]
        temp.append(list(score))

        #for i in range(len(score)):
        #    temp[i].append(score[i])
    col = ["ment_tw_v_{}".format(i) for i in range(10)]
    mnt_1 = pd.DataFrame(temp, columns=col)
    
    temp = [] 
    for word in dataFrame["mst_fr_ment_rt_word"]:
        if word[3:] == "@_":
            score = [0]*10
        else:
            score = model.wv[word[3:]]
        temp.append(list(score))

        #for i in range(len(score)):
        #    temp[i].append(score[i])
    col = ["ment_rt_v_{}".format(i) for i in range(10)]
    mnt_2 = pd.DataFrame(temp, columns=col)
    
    merged = pd.concat([dataFrame,hs_1,hs_2,mnt_1,mnt_2], axis="columns")
    
    merged = merged.drop(["mst_fr_hs_tw_word", 
                  "mst_fr_hs_rt_word",
                  "mst_fr_ment_tw_word",
                  "mst_fr_ment_rt_word"], axis="columns")
    return merged

def combine_data():
    df = pd.read_csv("../data/new_vectors_lexical.csv",sep = '\t', header=0)
    #df = pd.read_csv("../data/new_vectors_new.csv",sep = '\t', header=0)
    #df = pd.read_csv("new_vectors.csv",sep = '\t', header=0)
    #df = pd.read_csv("new_vectors_large.csv",sep = '\t', header=0)
    #df = pd.read_csv("new_vectors_large_extra.csv",sep = '\t', header=0)
    
    #Detach user ids feature and target from original DataFrame
    user_ids = df.iloc[:,-1]
    df = df.iloc[:,:-1]
    target = df.iloc[:,-1]
    df = df.iloc[:,:-1]
    
    #Read graph features 
    graph = [(np.int64(x.split("\"")[1]), np.int64(x.split("\"")[3])) for x in open("../data/ret_graph.dot", "r").read().split("\n") if "->" in x ]
    #identify self loops in graphs 
    self_loop = set([a for a,b in graph if a== b]) 
    
    #Get node degree data with weighted and un-weighted
    degree_data = { np.int64(x.split(",")[0]) : x.split(",")[3:] for x in open("../data/ret_data_degree.csv","r").read().split("\n")[1:] if x != ""}
    
    #Assemble all features in single DataFrame
    df["rt_self"]  = np.array([1 if x in self_loop else 0 for x in user_ids])
    df["in_degree"] = np.array([degree_data[x][0] if x in degree_data else 0 for x in user_ids ])
    df["out_degree"] = np.array([degree_data[x][1] if x in degree_data else 0 for x in user_ids ])
    df["w_in_degree"] = np.array([degree_data[x][3] if x in degree_data else 0 for x in user_ids ])
    df["w_out_degree"] = np.array([degree_data[x][4] if x in degree_data else 0 for x in user_ids ])
    df["w_degree"] = np.array([degree_data[x][5] if x in degree_data else 0 for x in user_ids ])
    



    #read labels from botometer and keep only users that agree to been bot or removed according to botometer
    
    filtered_bots = set([int(x.split(" ")[0]) for x in open("../data/bot_labels_jan.txt","r").read().split("\n") if x != ''])
    filtered_clear = set([int(x.split(" ")[0]) for x in open("../data/clear_labels_jan.txt","r").read().split("\n") if x != ''])

    keep_ids = []
    for x in np.where(target == 2)[0]:
        if user_ids[x]  in filtered_bots:
            keep_ids.append(x)
            target[x] = 1

    for x in np.where(target == 0)[0]:
        if user_ids[x] in filtered_clear:
            keep_ids.append(x)
            target[x] = 0
    
    df["target"] = target 
    df["user_id"] = user_ids
    
    print("Check for duplicates :{} and {}. Should be equeal".format(len(keep_ids), len(set(keep_ids))))
    #print(remove_ids)
    #print(len(remove_ids))
    #keep_ids = set(range(len(target))) - set(remove_ids)
    #target = target[list(keep_ids)]
    df = df.iloc[keep_ids]
    df.reset_index(drop=True, inplace=True)
    
    #make lexical columns
    
    to_drop = ['mst_fr_ment_tw_word','mst_fr_ment_rt_word','mst_fr_hs_tw_word','mst_fr_hs_rt_word']
    #to_drop = []
    #df_cp = df.copy()
    rep_1 = ["tw_#"+x for x in df.mst_fr_hs_tw_word]
    rep_2 = ["rt_#"+x for x in df.mst_fr_hs_rt_word]
    rep_3 = ["tw_@"+x for x in df.mst_fr_ment_tw_word]
    rep_4 = ["rt_@"+x for x in df.mst_fr_ment_rt_word]
    #display(rep_3)
    df["mst_fr_hs_tw_word"] = rep_1
    df["mst_fr_hs_rt_word"] = rep_2
    df["mst_fr_ment_tw_word"] = rep_3
    df["mst_fr_ment_rt_word"] = rep_4
    
    #perform one hot on textural features
    #df = one_hot(df)
    df = word_2_vec(df)
    

    df.to_csv (r'../data/features_large_lexical.csv', index = False, header=True)
    return df


In [3]:
#---------------------------------------------------------------------------
#FLAG -> 1: that can be executed at any machine with final csv file and 
#--------------------------------------------------------------------------

#balance DataFrame based on targets
def balance_data(data, labels, user_ids, verbose=False):
    labels_sizes = []
    for lb in set(labels):
        labels_sizes.append(len(np.where(labels == lb)[0]))
    min_size = min(labels_sizes)
    if verbose:
        print("Number of classes:{}".format(set(labels)))
        print("Size of minimum class:{}".format(min_size))
        print("End df size should be:{}".format(min_size*len(set(labels))))
    indexes = set()
    for lb in set(labels):
        indexes = indexes.union(set(random.sample(list(np.where(labels == lb)[0]), min_size)))
    #zero_group = random.sample(list(np.where(labels == 0)[0]), 10711)
    #one_group = random.sample(list(np.where(labels == 1)[0]), 10711)
    #two_group = list(np.where(labels == 2)[0])
    #print(zero_group)
    #print(len(zero_group))
    #print(len(one_group))
    #print(len(two_group))
    #print(data.iloc[two_group+zero_group+one_group,:].shape)
    
    data = data.iloc[list(indexes), :]
    labels = np.array(labels.iloc[list(indexes)])
    user_ids = np.array(user_ids.iloc[list(indexes)])
    
    #data = data.iloc[two_group+zero_group+one_group,:]
    #labels = np.array(labels.iloc[two_group+zero_group+one_group])
    #user_ids = np.array(user_ids.iloc[])
    """
    data = data.iloc[two_group+zero_group,:]
    labels = np.array(labels.iloc[two_group+zero_group])
    """
    return data, labels, user_ids

#one-hot
def one_hot(dataFrame):
    one_features=["mst_fr_hs_tw_word", 
                  "mst_fr_hs_rt_word",
                  "mst_fr_ment_tw_word",
                  "mst_fr_ment_rt_word"]
    
    #get dummies from features 
    dummies = [pd.get_dummies(dataFrame[f]) for f in one_features]
    
    #remove one extra columns for each dummies
    #in case when we have empty type like "tw_#_" remove it or choose the last one
    check_empty = ["tw_#_","rt_#_","tw_@_","rt_@_"]
    for i in range(len(dummies)):
        one_features.append(check_empty[i] if check_empty[i] in dummies[i].columns.tolist() else dummies[i].columns.tolist()[-1])
   
    #merge dummies and dataFrame and drop some unused features
    dummies.append(dataFrame)
    merged = pd.concat(dummies, axis="columns")
    merged = merged.drop(one_features, axis="columns")
    return merged

#load twitter features
def read_data(filename, verbose=False):
    #old parameters!
    #df = pd.read_csv("new_vectors.csv",sep = '\t', header=0)
    #df = pd.read_csv("features.csv", header=0)
    #df = pd.read_csv("features_large.csv", header=0)
    
    df = pd.read_csv(filename, header=0)
    
    #extract user id from dataframe
    user_ids = df["user_id"]
    
    #extract target from datafrmae
    target = df["target"]
    df = df.drop(["user_id","target"], axis="columns")
    
    #perform one hot on textural features
    #df = one_hot(df)
    #df = word_2_vec(df)
    
    #balance the data by labels (OVERSAMPLING)
    smk = SMOTETomek(random_state=42)
    data, target = smk.fit_sample(df,target)
    
    #data, target, user_ids = balance_data(df, target, user_ids,verbose=verbose)
    
    #scale features values with MinMaxScaler
    scaler = StandardScaler()
    df = scaler.fit(data).transform(data)
    df = pd.DataFrame(df, columns = data.columns)
    
    return df, target, None #user_ids

## Feature selection 

In [4]:
#---------------------------------------------------------------------------
#FLAG -> 1: that can be executed at any machine with final csv file and 
#--------------------------------------------------------------------------

def kbest_features(data, labels,n_features):
    #scaler = MinMaxScaler()
    #scaled_data = scaler.fit(data).transform(data)

    bestfeatures = SelectKBest(score_func=chi2, k=n_features)
    fit = bestfeatures.fit(data, labels)
    dfscores = pd.DataFrame(fit.scores_)
    dfcolumns = pd.DataFrame(data.columns)
    #concat two dataframes for better visualization 
    featureScores = pd.concat([dfcolumns,dfscores],axis=1)
    featureScores.columns = ['Specs','Score']  #naming the dataframe columns
    #print(featureScores.nlargest(n_features,'Score'))  #print 10 best features
    res = featureScores.nlargest(n_features,'Score')
    #print(dir(res))
    res = [x[0] for x in res.values]
    
    embeded_rf_support = [True if x in res else False for x in df.columns.tolist()]
    #res = [x[0] for x in res.values]
    #res = res.get_support()
    #embeded_rf_feature = data.loc[:,res].columns.tolist()
    return data[res]

def rforest_features(data, labels, n_features):
    #scaler = MinMaxScaler()
    #scaled_data = scaler.fit(data).transform(data)

    embeded_rf_selector = SelectFromModel(RandomForestClassifier(n_estimators=100), max_features=n_features)
    embeded_rf_selector.fit(data, labels)

    embeded_rf_support = embeded_rf_selector.get_support()
    embeded_rf_feature = data.loc[:,embeded_rf_support].columns.tolist()
    #print("R-Forest selected features: {} number of features: {}".format(embeded_rf_feature, len(embeded_rf_feature)))
    return data[embeded_rf_feature], embeded_rf_feature

def lasso_features(data, labels,n_features):
    #scaler = StandardScaler()
    #scaled_data= scaler.fit(data).transform(data)
    #embeded_lr_selector = SelectFromModel(LogisticRegression(solver = "sag", penalty="l2", max_iter=10000), 
    #                                      max_features=n_features)
    embeded_lr_selector = SelectFromModel(LogisticRegression(solver = "saga", penalty="l2", max_iter=10000), 
                                          max_features=n_features)
    
    embeded_lr_selector.fit(data, labels)
    embeded_lr_support = embeded_lr_selector.get_support()
    
    #print(embeded_lr_support)
    embeded_lr_feature = data.loc[:,embeded_lr_support].columns.tolist()
    #print("Lasso selected features: {} number of features:{}".format(embeded_lr_feature,len(embeded_lr_feature)))
    #return data[:,embeded_lr_support], embeded_lr_feature
    return data[embeded_lr_feature], embeded_lr_feature

def lasso(data, labels, alpha = 0.02):
    res = Lasso(alpha=alpha,selection='cyclic',max_iter=100000)
    res.fit(data,labels)
    mst = np.where(res.coef_ != 0.0)[0]
    #print(mst)
    #print("Features:{}".format(list(data.columns[mst])))
    #print("Number:{}".format(len(list(data.columns[mst]))))
    return data[list(data.columns[mst])], list(data.columns[mst])


# Dimentionality reduction - Clustering

In [ ]:
#---------------------------------------------------------------------------
#FLAG -> 1: that can be executed at any machine with final csv file and 
#--------------------------------------------------------------------------


colors = ['#e6194b', '#3cb44b', '#ffe119', '#4363d8', 
              '#f58231', '#911eb4', '#46f0f0', '#f032e6', 
              '#bcf60c', '#fabebe', '#008080', '#e6beff', 
              '#9a6324', '#fffac8', '#800000', '#aaffc3', 
              '#808000', '#ffd8b1', '#000075', '#808080', 
              '#ffffff', "#fe4a49", "#BD3430", "#fed766"]

#------------
#---T-SNE----
#------------
def make_tsne(data_scaled, label, title, learning_rate=100, perplexity=20):
    #tsne = TSNE(random_state=1)
    tsne = TSNE(learning_rate=learning_rate,perplexity=perplexity, init='pca')
    
    #use fit_transform instead of fit, as TSNE has no transform method
    digits_tsne = tsne.fit_transform(data_scaled)
    #create color palette
    palette = sns.color_palette(None, max(list(set(label))) + 1)
    plt.figure(figsize=(12.0,12.0))
    #print(len(palette))
    c_color = [palette[x] if x != -1 else (0,0,0) for x in label]
    
    plt.xlim(digits_tsne[:, 0].min(), digits_tsne[:, 0].max() + 1)
    plt.ylim(digits_tsne[:, 1].min(), digits_tsne[:, 1].max() + 1)
    plt.scatter(digits_tsne[:, 0],digits_tsne[:, 1],c=c_color)
    plt.gca().set_aspect('equal', 'datalim')
    plt.title('t-SNE projection {}'.format(title), fontsize=24)
    """
    for i in range(len(digits_tsne[:, 0])):
        # actually plot the digits as text instead of using scatter
        plt.text(digits_tsne[i, 0], digits_tsne[i, 1],str(target[i]),
                     color = colors[target[i]],
                     fontdict={'weight': 'bold', 'size': 9})
    """
    plt.xlabel("t-SNE feature 0")
    plt.xlabel("t-SNE feature 1")
    plt.show()
    return digits_tsne

#------------
#----UMAP----
#------------
def perofrm_umap_clustering(data, labels,  title, n_neighbors=25, n_components=2,):
    #reducer = umap.UMAP(n_neighbors=25,metric="manhattan")
    reducer = umap.UMAP(n_neighbors=n_neighbors, n_components= n_components, metric = "euclidean")#metric="euclidean")
    embedding = reducer.fit_transform(data)
    embedding.shape
    plt.figure(figsize=(10, 10))
    plt.scatter(
        embedding[:, 0],
        embedding[:, 1],
        c=[sns.color_palette()[x] for x in labels])
    plt.gca().set_aspect('equal', 'datalim')
    plt.title('UMAP projection {}'.format(title), fontsize=24)
    plt.show()
    return embedding

#------------
#---DBSCAN---
#------------
def dbscan_clustering(positions, eps, comp_filter, target, scale=False):
    #dbscan = DBSCAN(eps=0.0058)
    #dbscan = DBSCAN(eps=0.0050)
    #dbscan = DBSCAN(eps=0.0048)
    #dbscan = DBSCAN(eps=0.0049)
    dbscan = DBSCAN(eps=eps)
    if scale:
        scaler = MinMaxScaler()
        scaler.fit(positions)
        p_scaled = scaler.transform(positions)
    else:
        p_scaled = positions
    clusters = dbscan.fit_predict(p_scaled)
    
    #print("Cluster memberships:\n{}".format(clusters))
    #print("Unique slucter ids:{}\n\n".format(set(clusters)))
    #cmap = plt.cm.rainbow
    #norm = matplotlib.colors.Normalize(vmin=1.5, vmax=4.5)
    #print([(x/256.0, list(clusters).count(x)) for x in set(clusters)])
    #print(len(sns.color_palette()))
    #[print(sns.color_palette()[x]) for x in clusters]
    
    palette = sns.color_palette(None, len(set(clusters)))
    
    plt.figure(figsize=(12.0,12.0))

    c_color = [palette[x] if x != -1 else (0,0,0) for x in clusters]
    plt.scatter(p_scaled[:, 0], p_scaled[:, 1], color=c_color, cmap=mglearn.cm3, s=20)
    #plt.scatter(p_scaled[clusters == 0, 0], p_scaled[clusters == 0, 1], color=colors[0], cmap=mglearn.cm3, s=20)
    plt.xlabel("Feature 0")
    plt.ylabel("Feature 1")
    plt.show()
    
    components = defaultdict(lambda: set())
    bot_comp = defaultdict(lambda:set())
    clear_comp = defaultdict(lambda:set())
    comp_size = []
    """
    for i in range(len(clusters)):
        #if target[i] == 1:
        #    #print(clusters[i])
        comp_size.append(clusters[i])
    comp_size = set(comp_size)
    """
    #max_comp = max(set(comp_size), key = comp_size.count)
    #for i in range(len(clusters)):
    #    if clusters[i] in comp_size :
    #        #print(user_ids[i])
    #        c_o[clusters[i]].add(i)
    #        if target[i] == 1:
    #            b[clusters[i]].add(i)
    return clusters
    for i in range(len(clusters)):
        if clusters[i] == -1:
            continue
        components[clusters[i]].add(i)
        if target[i] == 1:
            bot_comp[clusters[i]].add(i)
        if target[i] == 0:
            clear_comp[clusters[i]].add(i)
    
    max_tp_bot = []
    for comp in bot_comp:
        if len(components[comp]) > comp_filter and len(bot_comp[comp])/len(components[comp]) > 0.5:
            max_tp_bot.append((len(bot_comp[comp])/len(components[comp])) *100.0)
            #print("TP :{} FP :{} Bperc: {} Comp size:{}".format((len(b[comp])/len(c_o[comp])) *100.0, ((len(c_o[comp]) - len(b[comp]))/len(c_o[comp])) * 100.0 , len(b[comp])/len(target[target == 1]), len(c_o[comp])))
    print("Bot accuracy Max:{} AVG:{} Min:{}".format(max(max_tp_bot), sum(max_tp_bot)/len(max_tp_bot) , min(max_tp_bot)))
    max_tp_clear = []
    for comp in clear_comp:
        if len(components[comp]) > comp_filter and len(clear_comp[comp])/len(components[comp]) > 0.5:
            max_tp_clear.append((len(clear_comp[comp])/len(components[comp])) *100.0)
    print("Clear accuracy Max:{} AVG:{} Min:{}".format(max(max_tp_clear), sum(max_tp_clear)/len(max_tp_clear) , min(max_tp_clear)))

#------------
#---DBSCAN--- !!!!!!with clustering sampling !!!!!!!
#------------
def dbscan_clustering_with_smaple(positions, eps, comp_filter, bot_indx, clear_indx, target=None):
    #dbscan = DBSCAN(eps=0.0058)
    #dbscan = DBSCAN(eps=0.0050)
    #dbscan = DBSCAN(eps=0.0048)
    #dbscan = DBSCAN(eps=0.0049)
    dbscan = DBSCAN(eps=eps)
    scaler = MinMaxScaler()
    scaler.fit(positions)
    p_scaled = scaler.transform(positions)
    clusters = dbscan.fit_predict(p_scaled)
    #print("Cluster memberships:\n{}".format(clusters))
    #print("Unique slucter ids:{}\n\n".format(set(clusters)))
    #cmap = plt.cm.rainbow
    #norm = matplotlib.colors.Normalize(vmin=1.5, vmax=4.5)
    #print([(x/256.0, list(clusters).count(x)) for x in set(clusters)])
    #print(len(sns.color_palette()))
    #[print(sns.color_palette()[x]) for x in clusters]
   
    palette = sns.color_palette(None, len(set(clusters)))
    
    plt.figure(figsize=(12.0,12.0))

    c_color = [palette[x] if x != -1 else (0,0,0) for x in clusters]
    plt.scatter(p_scaled[:, 0], p_scaled[:, 1], color=c_color, cmap=mglearn.cm3, s=20)
    #plt.scatter(p_scaled[clusters == 0, 0], p_scaled[clusters == 0, 1], color=colors[0], cmap=mglearn.cm3, s=20)
    plt.xlabel("Feature 0")
    plt.ylabel("Feature 1")
    plt.show()
    
    components = defaultdict(lambda: set())
    bot_comp = defaultdict(lambda:set())
    clear_comp = defaultdict(lambda:set())
    comp_size = []
    """
    for i in range(len(clusters)):
        #if target[i] == 1:
        #    #print(clusters[i])
        comp_size.append(clusters[i])
    comp_size = set(comp_size)
    """
    #max_comp = max(set(comp_size), key = comp_size.count)
    #for i in range(len(clusters)):
    #    if clusters[i] in comp_size :
    #        #print(user_ids[i])
    #        c_o[clusters[i]].add(i)
    #        if target[i] == 1:
    #            b[clusters[i]].add(i)
    comp_all = defaultdict(lambda: set())
    
    
    for i in range(len(clusters)):
        if clusters[i] == -1:
            continue
        comp_all[clusters[i]].add(i)
        if i in bot_indx or i in clear_indx:
            continue
        components[clusters[i]].add(i)
        if target[i] == 1:
            bot_comp[clusters[i]].add(i)
        if target[i] == 0:
            clear_comp[clusters[i]].add(i)
    
    max_tp = []
    max_fp = []
    bot_indx = set(bot_indx)
    clear_indx = set(clear_indx)
    
        
    for comp in bot_comp:
        #check comp size and purity 
        if len(components[comp]) > comp_filter and len(bot_comp[comp])/len(components[comp]) > 0.6:
            #at this point componnent is labeled as bot component
            
            sample_bot_comp = len(comp_all[comp].intersection(bot_indx))
            sample_clear_comp = len(comp_all[comp].intersection(clear_indx))
            if sample_bot_comp != 0 :
                max_tp.append( (sample_bot_comp / (sample_bot_comp+sample_clear_comp)) *100.0)
                print(comp)
                
            if sample_clear_comp != 0 :
                max_fp.append( (sample_clear_comp / (sample_clear_comp+sample_bot_comp)) *100.0 )
            #print("TP :{} FP :{} Bperc: {} Comp size:{}".format((len(b[comp])/len(c_o[comp])) *100.0, ((len(c_o[comp]) - len(b[comp]))/len(c_o[comp])) * 100.0 , len(b[comp])/len(target[target == 1]), len(c_o[comp])))
    plt.figure()
    for i in comp_all[9]:
        if i in bot_indx:
            plt.scatter(p_scaled[i, 0], p_scaled[i, 1], color = "#a11515")
        elif i in clear_indx:
            plt.scatter(p_scaled[i, 0], p_scaled[i, 1], color = "#a11515")
        elif i in clear_comp[1]:
            plt.scatter(p_scaled[i, 0], p_scaled[i, 1], c = "#167a19")
        elif i in bot_comp[1]:
            plt.scatter(p_scaled[i, 0], p_scaled[i, 1], c = "#a11515")
    plt.show()
    if len(max_tp) != 0:
        print("Bot TP accuracy Max:{} AVG:{} Min:{} len:{}".format(max(max_tp), sum(max_tp)/len(max_tp) , min(max_tp), len(max_tp)))
    else:
        print("Bot TP is Zero")
    if len(max_fp) != 0:
        print("Bot FP Max:{} AVG:{} Min:{} len:{}".format(max(max_fp), sum(max_fp)/len(max_fp) , min(max_fp), len(max_fp)))
    else:
        print("Bot FP is Zero")
    #max_tp_clear = []
    #for comp in clear_comp:
    #    if len(components[comp]) > comp_filter and len(clear_comp[comp])/len(components[comp]) > 0.9:
    #        max_tp_clear.append((len(clear_comp[comp])/len(components[comp])) *100.0)
    #print("Clear accuracy Max:{} AVG:{} Min:{}".format(max(max_tp_clear), sum(max_tp_clear)/len(max_tp_clear) , min(max_tp_clear)))

    
#------------
#-Mean-Shift- 
#------------
def mean_shift(data):
    scaler = MinMaxScaler()
    scaler.fit(data)
    p_scaled = scaler.transform(data)
    bandwidth = estimate_bandwidth(p_scaled, quantile=0.05, n_samples=500)
    meanshift = MeanShift(bandwidth=bandwidth)
    meanshift.fit(p_scaled)
    labels = meanshift.labels_
    labels_unique = np.unique(labels)
    n_clusters_ = len(labels_unique)
    P = meanshift.predict(p_scaled)
    palette = sns.color_palette(None, len(P))
    col = [palette[x] if x != -1 else (0,0,0) for x in P]
    plt.scatter(p_scaled[:,0], p_scaled[:,1], c=col, marker="o", picker=True)
    plt.title(f'Estimated number of clusters = {n_clusters_}')
    plt.xlabel('Temperature yesterday')
    plt.ylabel('Temperature today')
    plt.show()
    
    c_o = defaultdict(lambda: set())
    b = defaultdict(lambda:set())
    comp_size = []
    for i in range(len(P)):
        if target[i] == 1:
            #print(clusters[i])
            comp_size.append(P[i])
    comp_size = set(comp_size)
    
    #max_comp = max(set(comp_size), key = comp_size.count)
    for i in range(len(P)):
        if P[i] in comp_size :
            #print(user_ids[i])
            c_o[P[i]].add(i)
            if target[i] == 1:
                b[P[i]].add(i)
    for comp in c_o:
        if len(b[comp]) > 30 and len(c_o[comp]) > 50:
            print("TP :{} FP :{} Bperc: {}".format((len(b[comp])/len(c_o[comp])) *100.0, ((len(c_o[comp]) - len(b[comp]))/len(c_o[comp])) * 100.0 , len(b[comp])/len(target[target == 1]) ))

    
    
    


## Combine features (Database, Graph and new labels ) and store them into csv file 
### !!!!
### No need to run , execute next cell where data is readed from CSV file by filename
### !!!

In [62]:
#---------------------------------------------------------------------------
#FLAG  -> 0: that cannot be executed in any machine, excpet Alex.
#--------------------------------------------------------------------------


#---------------------!!!-------------------
#-----------------IMPORTANT-----------------
#Execute ONLY of case of data/labels changes
#-----------------IMPORTANT-----------------
#-------------------------------------------

df = combine_data()
display(df)

Check for duplicates :11836 and 11836. Should be equeal


,daily_rt_tw_0,daily_rt_tw_1,daily_rt_tw_2,daily_rt_tw_3,daily_rt_tw_4,daily_rt_tw_5,daily_rt_tw_6,daily_rt_0,daily_rt_1,daily_rt_2,daily_rt_3,daily_rt_4,daily_rt_5,daily_rt_6,daily_tw_0,daily_tw_1,daily_tw_2,daily_tw_3,daily_tw_4,daily_tw_5,daily_tw_6,hour_rt_tw_0,hour_rt_tw_1,hour_rt_tw_2,hour_rt_tw_3,hour_rt_tw_4,hour_rt_tw_5,hour_rt_tw_6,hour_rt_tw_7,hour_rt_tw_8,hour_rt_tw_9,hour_rt_tw_10,hour_rt_tw_11,hour_rt_tw_12,hour_rt_tw_13,hour_rt_tw_14,hour_rt_tw_15,hour_rt_tw_16,hour_rt_tw_17,hour_rt_tw_18,hour_rt_tw_19,hour_rt_tw_20,hour_rt_tw_21,hour_rt_tw_22,hour_rt_tw_23,hour_tw_0,hour_tw_1,hour_tw_2,hour_tw_3,hour_tw_4,hour_tw_5,hour_tw_6,hour_tw_7,hour_tw_8,hour_tw_9,hour_tw_10,hour_tw_11,hour_tw_12,hour_tw_13,hour_tw_14,hour_tw_15,hour_tw_16,hour_tw_17,hour_tw_18,hour_tw_19,hour_tw_20,hour_tw_21,hour_tw_22,hour_tw_23,hour_rt_0,hour_rt_1,hour_rt_2,hour_rt_3,hour_rt_4,hour_rt_5,hour_rt_6,hour_rt_7,hour_rt_8,hour_rt_9,hour_rt_10,hour_rt_11,hour_rt_12,hour_rt_13,hour_rt_14,hour_rt_15,hour_rt_16,hour_rt_17,hour_rt_18,hour_rt_19,hour_rt_20,hour_rt_21,hour_rt_22,hour_rt_23,mst_fr_ment_tw_1,mst_fr_ment_tw_2,mst_fr_ment_tw_3,mst_fr_ment_rt_1,mst_fr_ment_rt_2,mst_fr_ment_rt_3,mst_fr_hs_tw_1,mst_fr_hs_tw_2,mst_fr_hs_tw_3,mst_fr_hs_rt_1,mst_fr_hs_rt_2,mst_fr_hs_rt_3,tw_urls_avg,tw_urls_std,rt_urls_avg,rt_urls_std,tw_hash_avg,tw_hash_std,tw_ment_avg,tw_ment_std,rt_hash_avg,rt_hash_std,rt_ment_avg,rt_ment_std,rt_time_avg,rt_time_min,rt_time_max,rt_time_std,rt_avg,tw_avg,tw_rt_ration,verified,followers,favourites,listed,statuses,followers.1,friends_count,name_len,name_screen_sim,geo,protected,location,description,description_len,bckg_img,default_prof,entities,rt_self,in_degree,out_degree,w_in_degree,w_out_degree,w_degree,target,user_id,hs_tw_v_0,hs_tw_v_1,hs_tw_v_2,hs_tw_v_3,hs_tw_v_4,hs_tw_v_5,hs_tw_v_6,hs_tw_v_7,hs_tw_v_8,hs_tw_v_9,hs_rt_v_0,hs_rt_v_1,hs_rt_v_2,hs_rt_v_3,hs_rt_v_4,hs_rt_v_5,hs_rt_v_6,hs_rt_v_7,hs_rt_v_8,hs_rt_v_9,ment_tw_v_0,ment_tw_v_1,ment_tw_v_2,ment_tw_v_3,ment_tw_v_4,ment_tw_v_5,ment_tw_v_6,ment_tw_v_7,ment_tw_v_8,ment_tw_v_9,ment_rt_v_0,ment_rt_v_1,ment_rt_v_2,ment_rt_v_3,ment_rt_v_4,ment_rt_v_5,ment_rt_v_6,ment_rt_v_7,ment_rt_v_8,ment_rt_v_9
0,0.178988,0.112840,0.128405,0.070039,0.182879,0.128405,0.198444,0.187500,0.087500,0.062500,0.000000,0.187500,0.175000,0.300000,0.175141,0.124294,0.158192,0.101695,0.180791,0.107345,0.152542,0.0,0.000000,0.0,0.019455,0.015564,0.042802,0.035019,0.007782,0.038911,0.046693,0.054475,0.011673,0.019455,0.042802,0.073930,0.077821,0.066148,0.105058,0.116732,0.163424,0.054475,0.007782,0.000000,0.000000,0.0,0.0,0.000000,0.028249,0.00565,0.050847,0.045198,0.000000,0.028249,0.039548,0.056497,0.011299,0.022599,0.056497,0.084746,0.073446,0.067797,0.101695,0.129944,0.146893,0.045198,0.00565,0.00,0.00,0.0,0.000000,0.0,0.000000,0.0375,0.025,0.0125,0.025,0.0625,0.0625,0.05,0.012500,0.0125,0.012500,0.050000,0.087500,0.062500,0.112500,0.087500,0.2,0.075,0.0125,0.000000,0.000000,2.011174,5.980586,6.887675,2.011174,5.980586,9.177593,1.266133,6.667747,3.043134,1.266133,4.803376,6.667747,0.169492,0.411693,0.187500,0.433013,5.016949,5.633836,1.988701,2.665960,3.962500,4.602988,1.900000,2.915476,171.483958,0.166667,1292.233333,330.839721,2.758621,6.103448,2.212500,0,343,7349,0,3502,343,151,14,0.416667,0,0,0,1,86,1,1,0,1,115,0,576,166,742,1,1258962548503740425,3.493951,7.603045,-1.862264,10.015945,-0.595800,4.668619,3.420092,-1.148644,-2.369110,-4.642322,3.493951,7.603045,-1.862264,10.015945,-0.595800,4.668619,3.420092,-1.148644,-2.369110,-4.642322,0.282885,4.259559,4.080622,8.733304,-4.353861,4.428511,0.595667,-2.255075,-0.706327,-2.917101,0.282885,4.259559,4.080622,8.733304,-4.353861,4.428511,0.595667,-2.255075,-0.706327,-2.917101
1,0.000000,0.400000,0.000000,0.400000,0.000000,0.000000,0.200000,0.000000,0.250000,0.000000,0.500000,0.000000,0.000000,0.250000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.200000,0.000000,0.

## Execution main frame
### 1) Read CSV file into dataframe 
### 2) Balance data by target (50/50)
### 3) Perform One-Hot of text features
### As result, return dataframe, target of the dataframe and user_ids vector

In [65]:
#---------------------------------------------------------------------------
#FLAG -> 1: that can be executed at any machine with final csv file and 
#--------------------------------------------------------------------------
#used for testing of reading functions
#Read data from CSV file into dataframe with balancing by target values and one-hot of text features

"""
#old file without any text features
filename = "features_large_extra.csv"
"""
filename = "../data/features_large_lexical.csv"

df, target, user_ids = read_data(filename,verbose=True)

print(df.shape)
display(df)

(13142, 187)


,daily_rt_tw_0,daily_rt_tw_1,daily_rt_tw_2,daily_rt_tw_3,daily_rt_tw_4,daily_rt_tw_5,daily_rt_tw_6,daily_rt_0,daily_rt_1,daily_rt_2,daily_rt_3,daily_rt_4,daily_rt_5,daily_rt_6,daily_tw_0,daily_tw_1,daily_tw_2,daily_tw_3,daily_tw_4,daily_tw_5,daily_tw_6,hour_rt_tw_0,hour_rt_tw_1,hour_rt_tw_2,hour_rt_tw_3,hour_rt_tw_4,hour_rt_tw_5,hour_rt_tw_6,hour_rt_tw_7,hour_rt_tw_8,hour_rt_tw_9,hour_rt_tw_10,hour_rt_tw_11,hour_rt_tw_12,hour_rt_tw_13,hour_rt_tw_14,hour_rt_tw_15,hour_rt_tw_16,hour_rt_tw_17,hour_rt_tw_18,hour_rt_tw_19,hour_rt_tw_20,hour_rt_tw_21,hour_rt_tw_22,hour_rt_tw_23,hour_tw_0,hour_tw_1,hour_tw_2,hour_tw_3,hour_tw_4,hour_tw_5,hour_tw_6,hour_tw_7,hour_tw_8,hour_tw_9,hour_tw_10,hour_tw_11,hour_tw_12,hour_tw_13,hour_tw_14,hour_tw_15,hour_tw_16,hour_tw_17,hour_tw_18,hour_tw_19,hour_tw_20,hour_tw_21,hour_tw_22,hour_tw_23,hour_rt_0,hour_rt_1,hour_rt_2,hour_rt_3,hour_rt_4,hour_rt_5,hour_rt_6,hour_rt_7,hour_rt_8,hour_rt_9,hour_rt_10,hour_rt_11,hour_rt_12,hour_rt_13,hour_rt_14,hour_rt_15,hour_rt_16,hour_rt_17,hour_rt_18,hour_rt_19,hour_rt_20,hour_rt_21,hour_rt_22,hour_rt_23,mst_fr_ment_tw_1,mst_fr_ment_tw_2,mst_fr_ment_tw_3,mst_fr_ment_rt_1,mst_fr_ment_rt_2,mst_fr_ment_rt_3,mst_fr_hs_tw_1,mst_fr_hs_tw_2,mst_fr_hs_tw_3,mst_fr_hs_rt_1,mst_fr_hs_rt_2,mst_fr_hs_rt_3,tw_urls_avg,tw_urls_std,rt_urls_avg,rt_urls_std,tw_hash_avg,tw_hash_std,tw_ment_avg,tw_ment_std,rt_hash_avg,rt_hash_std,rt_ment_avg,rt_ment_std,rt_time_avg,rt_time_min,rt_time_max,rt_time_std,rt_avg,tw_avg,tw_rt_ration,verified,followers,favourites,listed,statuses,followers.1,friends_count,name_len,name_screen_sim,geo,protected,location,description,description_len,bckg_img,default_prof,entities,rt_self,in_degree,out_degree,w_in_degree,w_out_degree,w_degree,hs_tw_v_0,hs_tw_v_1,hs_tw_v_2,hs_tw_v_3,hs_tw_v_4,hs_tw_v_5,hs_tw_v_6,hs_tw_v_7,hs_tw_v_8,hs_tw_v_9,hs_rt_v_0,hs_rt_v_1,hs_rt_v_2,hs_rt_v_3,hs_rt_v_4,hs_rt_v_5,hs_rt_v_6,hs_rt_v_7,hs_rt_v_8,hs_rt_v_9,ment_tw_v_0,ment_tw_v_1,ment_tw_v_2,ment_tw_v_3,ment_tw_v_4,ment_tw_v_5,ment_tw_v_6,ment_tw_v_7,ment_tw_v_8,ment_tw_v_9,ment_rt_v_0,ment_rt_v_1,ment_rt_v_2,ment_rt_v_3,ment_rt_v_4,ment_rt_v_5,ment_rt_v_6,ment_rt_v_7,ment_rt_v_8,ment_rt_v_9
0,-0.049878,-0.133175,0.255009,-0.443937,0.278000,0.004676,0.237817,0.056613,-0.203744,-0.181365,-0.644724,0.414743,0.378305,0.817755,0.593237,0.353672,0.880367,0.188557,0.697298,0.267648,0.529351,-0.435038,-0.452749,-0.442637,-0.239920,-0.186539,0.214214,0.237441,-0.081292,0.410983,0.559996,0.489622,-0.181067,-0.192212,-0.051671,0.190479,0.176762,0.091783,0.429418,0.520992,0.940754,0.053540,-0.359803,-0.436537,-0.448425,-0.234270,-0.237339,-0.231494,0.113272,-0.100479,0.658536,0.776964,-0.100883,0.491334,0.869011,0.859229,0.015466,0.063923,0.366716,0.602564,0.409162,0.349641,0.726576,1.072443,1.234302,0.229552,-0.168180,-0.225899,-0.231273,-0.378525,-0.400362,-0.391869,-0.352316,0.054751,0.031453,-0.045766,0.198202,0.782460,0.849837,0.436933,-0.139369,-0.204228,-0.244927,0.047755,0.312968,0.144492,0.575189,0.332462,1.344492,0.301904,-0.255169,-0.381947,-0.382997,-0.085451,0.941694,1.278872,-0.394640,0.664480,1.578557,-0.016925,1.435689,0.383403,-0.504084,0.504676,1.039012,-0.017361,0.524030,-0.371804,0.006522,2.705896,2.768277,0.779003,0.911884,1.291034,1.271117,0.474262,0.512145,-0.514654,-0.432734,1.172767,-0.180697,2.036184,5.926834,0.176183,-0.185022,-0.064789,-0.560319,-0.104502,-0.551542,-0.064789,-0.391907,-0.016518,-0.205231,-0.626345,0.0,-1.276294,0.444843,-0.135923,0.428661,0.750604,-0.446928,3.831785,0.069525,-0.514436,0.114732,1.635110,0.146035,1.309026,1.626375,-0.810770,1.458216,-0.658625,1.603797,1.069426,-0.705271,-0.413639,-1.271852,0.906266,0.807851,-0.834808,0.690454,-0.776477,0.931283,0.555655,-0.540717,0.115497,-0.308036,0.271886,1.966123,1.839228,2.912880,-1.706381,2.113599,0.160782,-1.336405,-0.300694,-1.764599,0.342613,0.900884,0.527357,1.220260,-0.833783,0.817196,-0.067509,-0.740512,-0.274670,-0.462836
1,-0.775159,1.868781,-0.796975,1.005527,-0.694484,-0.680066,0.

## K-Fold Cross Validation Version 2 (manual implementation of K-fold)
 - Select features at each fold based by train split , each iteration select new features
 - at the end of execution make stratified 70/30 split with union of all selected features

In [66]:
#---------------------------------------------------------------------------
#FLAG -> 1: that can be executed at any machine with final csv file and 
#--------------------------------------------------------------------------

#read csv file data with oversampling
filename = "../data/features_large_lexical.csv"
df, target, user_ids = read_data(filename)

def k_fold_val_2(number_of_features, objective="multi:softprob",
                               num_class = 2,
                               learning_rate =0.2,
                               n_estimators=100,
                               max_depth=5,
                               colsample_bytree = 0.7,
                               eval_metric="aucpr"#"rmse",
                                ):
    accur_scores = []
    #create Strafied 5 fold class
    kf = StratifiedKFold(n_splits=5,shuffle=True)
    all_features = None
    
    #print("Alpha:{:.6f}".format(number_of_features))
    
    #make 5 fold cross valdation with use of features selected by train split
    
    for train_index, test_index in kf.split(df,target):
        X_train = df.iloc[train_index]
        X_train, X_test, y_train, y_test = df.iloc[train_index], df.iloc[test_index], target[train_index], target[test_index]

        #select K features with use of random forest feature selection
        X_train_f, ft = rforest_features(X_train, y_train, number_of_features)
        #X_train_f, ft = lasso(X_train, y_train, alpha=number_of_features)
        if all_features == None:
            all_features = set(ft)
        else:
            all_features = all_features.intersection(set(ft))
        #for f in ft:
        #    all_features.append(f)

        #keep only selected features in train and test splits
        X_test_f = X_test[ft]
        """
        #another classifiers for cross validation, that have pour performance till now
        reg_scores = cross_val_score(LogisticRegression(max_iter=10000,penalty='l1',
                                                        solver='liblinear',C=1.0), X, target,cv=5)
        
        svc_scores = cross_val_score(SVC(kernel='rbf', C=0.9), X, target,cv=5)
        
        for_scores = cross_val_score(RandomForestClassifier(n_estimators=35,
                                                            criterion='gini',
                                                            ccp_alpha=0.0003,
                                                            min_samples_split=4), X, target,cv=5)

        """
        xGb_cl = XGBClassifier(objective=objective,
                               num_class = num_class,
                               learning_rate =learning_rate,
                               n_estimators=n_estimators,
                               max_depth=max_depth,
                               colsample_bytree = colsample_bytree,
                               eval_metric=eval_metric,#"rmse",
                               use_label_encoder=False)
        #fit model with train dataset
        xGb_cl.fit(X_train_f, y_train)

        #predict test dataset and keep prediction labels
        pred = xGb_cl.predict(X_test_f)
        #print("Fold accuracy score: ",accuracy_score(y_test, pred))
        accur_scores.append(accuracy_score(y_test, pred))
    #print(all_features)
    accur_scores = sum(accur_scores) / len(accur_scores)
    #print("Avg k-fold:{} for :{} features".format(accur_scores, number_of_features))
    #plt.figure(num=None, figsize=(60, 60))
    list_features = list(set(all_features))
    #print(len(list_features))
    #print(len(ft))
    
    #count_features = [all_features.count(f) for f in list_features]
    #print(count_features)
    #plt.bar(range(len(list_features)), count_features)
    #plt.xticks(range(len(list_features)), list_features, rotation=45)
    #plt.savefig("features.png",dpi=600)

    #after k-folds keep all selected features and make 70/30 stratified split and train and predict
    all_features = list(set(all_features))
    #print("70/30")
    """
    X_train, X_test, y_train, y_test = train_test_split(df[all_features], 
                                                        target,
                                                        test_size=0.3,
                                                        stratify=target)

    xGb_cl = XGBClassifier(objective="multi:softprob",
                           num_class = 2,
                           learning_rate =0.2,
                           n_estimators=100,
                           max_depth=5,
                           colsample_bytree = 0.7,
                           eval_metric="aucpr",#"rmse",
                           use_label_encoder=False)

    xGb_cl.fit(X_train, y_train)
    pred = xGb_cl.predict(X_test)
    print("Final accuracy score:",accuracy_score(y_test, pred),"\n--------------\n")
    return accur_scores, accuracy_score(y_test, pred)
    """
    return accur_scores

#k_fold_val_2(number_of_features=80,n_rep=1)

## Parameter fine tuning

In [52]:
#---------------------------------------------------------------------------
#FLAG -> 1: that can be executed at any machine with final csv file and 
#--------------------------------------------------------------------------


"""
learning_rate =[0.1,0.2,0.3,0.4]
n_estimators=[50,100, 150,200,250,300,350]
max_depth=[1,2,3,4,5,6,7,8,9,10,15,20]
colsample_bytree = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]
eval_metric=["rmse","auc"]
"""

print(df.shape)
values = list(np.arange(0.00080, 0.002, 0.00001))
values=list(range(40,50))
#X,ft = lasso(df, target, alpha=0.02)
#print(len(ft))

plt.figure(figsize=(8.0,8.0))
k_scores = defaultdict(lambda: 0)

objective=["multi:softprob"]
learning_rate =[0.1, 0.2, 0.3, 0.4]
n_estimators=[100, 150, 200]
max_depth=[4,5,6,7,8]
colsample_bytree = [0.6, 0.7, 0.8, 0.9]
eval_metric=["aucpr","rmse"]

for lr in learning_rate:
    for n_est in n_estimators:
        for m_depth in max_depth:
            for bytree in colsample_bytree:
                for ev_m in eval_metric:
                    k_scores = []
                    f_scores = []
                    for it in range(5):
                        k_accuracy = k_fold_val_2(number_of_features=45, objective="multi:softprob",
                               num_class = 2,
                               learning_rate =lr,
                               n_estimators=n_est,
                               max_depth=m_depth,
                               colsample_bytree = bytree,
                               eval_metric=ev_m)
                        #k_accuracy, full_accuracy = k_fold_val_2(number_of_features=alpha)
                        k_scores.append(k_accuracy)
                        #f_scores.append(full_accuracy)
                    
                    if sum(k_scores)/len(k_scores) >= 0.938:
                        print("LR: {}\nnEst: {}\nm_depth: {}\nbytree: {}\nev_m: {} score avg:{}".format(
                                        lr,
                                        n_est, 
                                        m_depth,
                                        bytree, 
                                        ev_m, 
                                        sum(k_scores)/len(k_scores) ) )

#res = []
#for alpha in values:
#    res.append(k_scores[alpha] / 10)
#plt.plot(values, res, label="K-Fold")
#plt.plot(values,f_scores, label="Full Split 70/30")
#plt.legend()
#plt.xticks(rotation=45)
#plt.savefig("Lasso_feature.png",dpi=600)


#features = list(range(2,167))
#accuracy_fold = []
#accuracy_split = []
#for f_number in features:
#    acc_s, acc_s_s = k_fold_val_2(number_of_features=f_number,n_rep=10)
#    accuracy_fold.append(acc_s)
#    accuracy_split.append(acc_s_s)
#plt.plot(features, accuracy_fold, label="K-Fold avg accuracy")
#plt.plot(features, accuracy_split, label="70/30 split accuracy")


(13142, 187)
LR: 0.1
nEst: 100
m_depth: 4
bytree: 0.6
ev_m: rmse score avg:0.938289162039377
LR: 0.1
nEst: 100
m_depth: 5
bytree: 0.6
ev_m: aucpr score avg:0.9381370997763501
LR: 0.1
nEst: 100
m_depth: 5
bytree: 0.8
ev_m: rmse score avg:0.938274143973118
LR: 0.1
nEst: 100
m_depth: 5
bytree: 0.9
ev_m: rmse score avg:0.9384722041299105
LR: 0.1
nEst: 100
m_depth: 6
bytree: 0.6
ev_m: rmse score avg:0.9381825534533735
LR: 0.1
nEst: 100
m_depth: 7
bytree: 0.7
ev_m: aucpr score avg:0.9383656650183848
LR: 0.1
nEst: 100
m_depth: 7
bytree: 0.8
ev_m: aucpr score avg:0.9381829992479387
LR: 0.1
nEst: 100
m_depth: 8
bytree: 0.6
ev_m: aucpr score avg:0.9380156525998217
LR: 0.1
nEst: 100
m_depth: 8
bytree: 0.8
ev_m: rmse score avg:0.9381980925782152
LR: 0.1
nEst: 100
m_depth: 8
bytree: 0.9
ev_m: aucpr score avg:0.938152210475246
LR: 0.1
nEst: 150
m_depth: 4
bytree: 0.8
ev_m: rmse score avg:0.9381827387186474
LR: 0.1
nEst: 150
m_depth: 6
bytree: 0.7
ev_m: rmse score avg:0.9384262062361449
LR: 0.1
nEst:

KeyboardInterrupt: 

<Figure size 576x576 with 0 Axes>

### Dimentionality reduction t-SNE

In [ ]:
#---------------------------------------------------------------------------
#FLAG  -> 0: that cannot be executed in any machine, excpet Alex.
#--------------------------------------------------------------------------


#X, _ = lasso_features(df, target, 4)
#X,_ = lasso(df, target, alpha=0.00002)
X = rforest_features(df, target, 56)
tsne_2d = make_tsne(X, target, "full dataset",learning_rate=100, perplexity=20)
#tsne_2d = make_tsne(X, target, "full dataset")
#tsne_2d = make_tsne(X, target, "full dataset")
#print("Plot t-SNE for train dataset with features selected from train portion")
#make_tsne(X_train_scaled, y_train, "train dataset")
#print("Plot t-SNE for test dataset with features selected from train portion")
#make_tsne(X_test_scaled, y_test, "test dataset")

### Dimentionality reduction UMAP

In [ ]:
#---------------------------------------------------------------------------
#FLAG  -> 0: that cannot be executed in any machine, excpet Alex.
#--------------------------------------------------------------------------

#X, _ = lasso_features(df, target, 4)
umap_2d = perofrm_umap_clustering(X, target, "all dataset")
#embedding = perofrm_umap_clustering(X, target, "all dataset")
#embedding = perofrm_umap_clustering(X, target, "all dataset")
#perofrm_umap_clustering(X_test_scaled, y_test,  "test dataset")
#perofrm_umap_clustering(X_train_scaled, y_train, "train dataset")

### Clustering DBSCAN

In [ ]:
#---------------------------------------------------------------------------
#FLAG  -> 0: that cannot be executed in any machine, excpet Alex.
#--------------------------------------------------------------------------

#dbscan_clustering(tsne_2d, 0.00557, 25)
#dbscan_clustering(tsne_2d, 0.0045, 25)
#dbscan_clustering(tsne_2d, 0.0057, 30)
#dbscan_clustering(umap_2d,0.007, 30)
#dbscan_clustering(umap_2d,0.0025, 20)
dbscan_clustering(umap_2d,0.0025, 20)
#dbscan_clustering(X, 0.00557, 25)


## Mean-Shift 

In [ ]:
#---------------------------------------------------------------------------
#FLAG  -> 0: that cannot be executed in any machine, excpet Alex.
#--------------------------------------------------------------------------

mean_shift(tsne_2d)

## Cosine Similarity

In [ ]:
#---------------------------------------------------------------------------
#FLAG  -> 0: that cannot be executed in any machine, excpet Alex.
#--------------------------------------------------------------------------


res = cosine_similarity(X,Y= None, dense_output=True)
pairs = set()
f_out = open("cosine_graph.dot","w+")
f_out.write("digraph {\n")
for i in range(0, res.shape[0]):
    f_out.write("\t\"{}\"     [label=\"{}\"];\n".format(i,i))
    #    "1" -- "2" [weight=100];")
for i in range(0,res.shape[0]):
    max_sim = res[i,list(set(range(0,39123)) - set([i]))]
    max_sim = [x for x in max_sim if x >= 0.9]
    if len(max_sim) == 0:
        continue
    max_sim.sort(reverse=True)
    if len(max_sim) > 5:
        max_sim = max_sim[:5]
    
    
    #print(max_sim)
    #print(np.where(res[1,:] == max_sim))
    for m_value in max_sim:
        line = "\t\"{}\" -- \"{}\" [weight={}];\n".format(i, np.where(res[i,:] == m_value)[0][0], int(m_value*100.0))
        line_rev = "\t\"{}\" -- \"{}\" [weight={}];\n".format(np.where(res[i,:] == m_value)[0][0], i, int(m_value*100.0))
        if line in pairs or line_rev in pairs:
            continue
        pairs.add(line)
        #print(line)
        f_out.write(line)
f_out.write("}")
f_out.close()

## GaussianMixture

In [ ]:
#---------------------------------------------------------------------------
#FLAG  -> 0: that cannot be executed in any machine, excpet Alex.
#--------------------------------------------------------------------------


gmm = GaussianMixture(n_components=2,n_init=100,init_params = "kmeans",covariance_type = "full")
#X, _ = lasso_features(df, target, 10)
#X = kbest_features(df, target, 3)
gmm.fit(X)

#predictions from gmm
labels = gmm.predict(X)
TP = [True if labels[i]==1 and target[i] == 1 else False for i in range(len(target)) ].count(True)
TN = [True if labels[i]==0 and target[i] == 0 else False for i in range(len(target)) ].count(True)
FP = [True if labels[i]==1 and target[i] == 0 else False for i in range(len(target)) ].count(True)
FN = [True if labels[i]==0 and target[i] == 1 else False for i in range(len(target)) ].count(True)
P = len(target[target == 1])
N = len(target[target == 0])
print("True pos :{} tp rate:{}".format(TP,TP/P))
print("True neg :{} tn rate:{}".format(TN, TN/N))
print("False pos :{} fp rate:{}".format(FP,FP/N))
print("False neg :{} fn rate:{}".format(FN,FN/P))

In [ ]:
#---------------------------------------------------------------------------
#FLAG  -> 0: that cannot be executed in any machine, excpet Alex.
#--------------------------------------------------------------------------

#X = lasso_features(df, target, 4)
#X = rforest_features(df, target, 4)
TPR_L = []
TNR_L = []
FNR_L = []
FPR_L = []
purity = []
#for n_features in range(1,10):
for j in range(2,40):
    #X = kbest_features(df, target, n_features)
    X, _ = lasso_features(df, target, 4)
    #X = rforest_features(df, target, 4)
   
    TPR = []
    TNR = []
    FPR = []
    FNR = []
    tmp_pur = []
    for k in range(0,10):
        gmm = GaussianMixture(n_components=j,init_params = "kmeans",covariance_type = "full")
        gmm.fit(X)

        #predictions from gmm
        labels = gmm.predict(X)
        tmp_pur.append(compute_pur(labels,target))
        TP = [True if labels[i]==1 and target[i] == 2 else False for i in range(len(target)) ].count(True)
        TN = [True if labels[i]==0 and target[i] == 0 else False for i in range(len(target)) ].count(True)
        FP = [True if labels[i]==1 and target[i] == 0 else False for i in range(len(target)) ].count(True)
        FN = [True if labels[i]==0 and target[i] == 2 else False for i in range(len(target)) ].count(True)
        P = len(target[target == 2])
        N = len(target[target == 0])
        TPR.append(TP/P)
        TNR.append(TN/N)
        FPR.append(FP/N)
        FNR.append(FN/P)
    purity.append(sum(tmp_pur) / len(tmp_pur))
    TPR = sum(TPR) / len(TPR)
    TNR = sum(TNR) / len(TNR)
    FPR = sum(FPR) / len(FPR)
    FNR = sum(FNR) / len(FNR)
    
    #print("Number of features:{} TPR:{} TNR:{} FNR:{} FPR:{}".format(n_features, TPR, TNR, FNR, FPR))
    TPR_L.append(TPR)
    TNR_L.append(TNR)
    FNR_L.append(FNR)
    FPR_L.append(FPR)
plt.figure(figsize=(12.0,12.0))
plt.plot(range(1,len(TPR_L)+1) ,TPR_L, label="True Positive", color="Blue" )
plt.plot(range(1,len(TNR_L)+1) ,TNR_L, label="True Negative", color="Green" )
plt.plot(range(1,len(FNR_L)+1) ,FNR_L, label="False Negative", color="Red")
plt.plot(range(1,len(FPR_L)+1) ,FPR_L, label="False Positive" , color ="y")
plt.xticks(range(1,len(FPR_L)+1))
plt.xlabel("Number of cluster")
plt.legend()
plt.show()
plt.clf()
plt.figure(figsize=(12.0,12.0))
plt.plot(range(2,40),purity, label="Purity avg")
plt.title("Avg cluster purity")
plt.show()
print(1+TPR_L.index(max(TPR_L)))
print(1+TNR_L.index(max(TNR_L)))
print(1+FNR_L.index(min(FNR_L)))
print(1+FPR_L.index(min(FPR_L)))


    
    


### Anomaly detection


In [ ]:
#---------------------------------------------------------------------------
#FLAG  -> 0: that cannot be executed in any machine, excpet Alex.
#--------------------------------------------------------------------------



X, _ = lasso_features(df, target, 4)
X_s = X[target==0,:]
print(X_s.shape)
cov = EllipticEnvelope().fit(X)
res = cov.predict(X)
print(len(target))
labels = [target[i] for i in range(len(res)) if res[i] == -1]

print("Wrong:",labels.count(0))
print("Right:",labels.count(2))
gm = GaussianMixture(n_components=2)
gm.fit(X)
densities = gm.score_samples(X)
print(len(densities))
density_threshold = np.percentile(densities, 4)
anomalies = np.where(densities < density_threshold)[0]
labels = [target[i] for i in anomalies]
print(labels.count(0))
print(labels.count(2))
#print(anomalies)

In [ ]:
#---------------------------------------------------------------------------
#FLAG  -> 0: that cannot be executed in any machine, excpet Alex.
#--------------------------------------------------------------------------


from sklearn.mixture import BayesianGaussianMixture
X, _ = lasso_features(df, target, 4)
print(set(target))
bgm = BayesianGaussianMixture(n_components=3, n_init=10, random_state=0,covariance_type='tied',init_params='kmeans')
bgm.fit(X)
labels = bgm.predict(X)

labels[target == 2]

TP = [True if labels[i]==2 and target[i] == 2 else False for i in range(len(target)) ].count(True)
TN = [True if labels[i]==0 and target[i] == 0 else False for i in range(len(target)) ].count(True)
FP = [True if labels[i]==2 and target[i] == 0 else False for i in range(len(target)) ].count(True)
FN = [True if labels[i]==0 and target[i] == 2 else False for i in range(len(target)) ].count(True)
P = len(target[target == 2])
N = len(target[target == 0])
print("True pos :{} tp rate:{}".format(TP,TP/P))
print("True neg :{} tn rate:{}".format(TN, TN/N))
print("False pos :{} fp rate:{}".format(FP,FP/N))
print("False neg :{} fn rate:{}".format(FN,FN/P))



In [ ]:
#---------------------------------------------------------------------------
#FLAG  -> 0: that cannot be executed in any machine, excpet Alex.
#--------------------------------------------------------------------------


# calculate interquartile range
def remove_outliers(data, labels):
    all_outliers = set()
    for i in range(data.shape[1]):
        q25, q75 = np.percentile(data[:,i], 25), np.percentile(data[:,i], 75)
        iqr = q75 - q25
        # calculate the outlier cutoff
        cut_off = iqr * 1.5
        lower, upper = q25 - cut_off, q75 + cut_off
        # identify outliers
        outliers = set()
        for x in data[:,i]:
            if x < lower or x > upper:
                outliers = set(np.where(data[:,i] == x)[0]).union(outliers)
        #outliers = [np.where(X[:,i] == x)[0] for x in X[:,i] if x < lower or x > upper]
        #print(len(outliers))
        #print(len(set(outliers)))
        all_outliers = all_outliers.union(outliers)
    #print("----")
    #print(len(all_outliers))
    l = [labels[i] for i in all_outliers]
    #print(l.count(0))
    #print(l.count(1))
    #print(l.count(2))
    ind_clear = list(set(range(X.shape[0])) -all_outliers)
    X_clear = data[ind_clear,:]
    target_clear = labels[ind_clear]
    #print(X_clear.shape)
    return X_clear, target_clear
#print(len(outliers))
#print(target[outliers])

In [ ]:
#---------------------------------------------------------------------------
#FLAG  -> 0: that cannot be executed in any machine, excpet Alex.
#--------------------------------------------------------------------------

from sklearn.mixture import BayesianGaussianMixture
#X, _ = lasso_features(df, target, 4)
print(set(target))
#bgm = BayesianGaussianMixture(n_components=3, n_init=10, random_state=0,covariance_type='tied',init_params='kmeans')
bgm = GaussianMixture(n_components=3, n_init=10,max_iter=1000, random_state=0,covariance_type = "full",init_params='kmeans')

bgm.fit(X_clear)
labels = bgm.predict(X_clear)

print(labels[target_clear == 2])

TP = [True if labels[i]==2 and target_clear[i] == 2 else False for i in range(len(target_clear)) ].count(True)
TN = [True if labels[i]==0 and target_clear[i] == 0 else False for i in range(len(target_clear)) ].count(True)
FP = [True if labels[i]==2 and target_clear[i] == 0 else False for i in range(len(target_clear)) ].count(True)
FN = [True if labels[i]==0 and target_clear[i] == 2 else False for i in range(len(target_clear)) ].count(True)
P = len(target_clear[target_clear == 2])
N = len(target_clear[target_clear == 0])
print("True pos :{} tp rate:{}".format(TP,TP/P))
print("True neg :{} tn rate:{}".format(TN, TN/N))
print("False pos :{} fp rate:{}".format(FP,FP/N))
print("False neg :{} fn rate:{}".format(FN,FN/P))

In [ ]:
#---------------------------------------------------------------------------
#FLAG  -> 0: that cannot be executed in any machine, excpet Alex.
#--------------------------------------------------------------------------


r = labels[target_clear == 2]
print(len(np.where(r == 0)[0]))
print(len(np.where(r == 1)[0]))
print(len(np.where(r == 2)[0]))
print("------------")
r = labels[target_clear == 1]
print(len(np.where(r == 0)[0]))
print(len(np.where(r == 1)[0]))
print(len(np.where(r == 2)[0]))
print("------------")
r = labels[target_clear == 0]
print(len(np.where(r == 0)[0]))
print(len(np.where(r == 1)[0]))
print(len(np.where(r == 2)[0]))

In [ ]:
#---------------------------------------------------------------------------
#FLAG  -> 0: that cannot be executed in any machine, excpet Alex.
#--------------------------------------------------------------------------

df, target, user_ids = read_data("features_large_extra.csv")
#new_lb = { int(i.split(" ")[0]) : int(i.split(" ")[1]) for i in open('labels_sample.txt','r').read().split("\n") if i != ''}
#new_target = np.array([new_lb[user_ids[u]] for u in range(len(user_ids))])
#print(new_target.shape)
print(target.shape)
#X, _ = lasso_features(df, target, 5)
#X = rforest_features(df, target, 5)
#X = kbest_features(df, new_target, 5)
#X_clear, target_clear = remove_outliers(X, target)
#clusters = dbscan_clustering(X_clear, 0.03, 25)
#print(set(clusters))
#cc = defaultdict(lambda: set())
#for i in range(len(clusters)):
#    cc[clusters[i]].add(i)
#g_1 = set(np.where(target_clear == 1)[0])
#g_2 = set(np.where(target_clear == 2)[0])
#g_0 = set(np.where(target_clear == 0)[0])
#for c in cc:
#    if c == -1:
#        continue
#    print(" Inter g_0: {} g_1: {} g_2: {}".format(len(g_0.intersection(cc[c])),  len(g_1.intersection(cc[c])), len(g_2.intersection(cc[c])) ))
    

for f in range(1,10):
    bic = []
    aic=[]
    plt.figure(figsize=(12.0,12.0))
    X, _ = lasso_features(df, target, f)
    print("Features:{}".format(f))
    for n in range(1,50):
        bgm = GaussianMixture(n_components=n, n_init=10,max_iter=10000, random_state=0,covariance_type = "tied",init_params='kmeans')

        bgm.fit(X)
        bic.append(bgm.bic(X))
        aic.append(bgm.aic(X))
    plt.plot(range(len(bic)), bic,label="Bic")
    plt.plot(range(len(aic)), aic,label="Aic")
    plt.legend()
    plt.savefig("GM_f_{}_tied.png".format(f))
    plt.clf()
#accuracy_score(target, labels)

In [ ]:
#---------------------------------------------------------------------------
#FLAG  -> 0: that cannot be executed in any machine, excpet Alex.
#--------------------------------------------------------------------------


from sklearn import datasets
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

import matplotlib.pyplot as plt
df, target, user_ids = read_data("features_large_extra.csv")

print(target.shape)
#X, _ = lasso_features(df, target, 5)
#X = rforest_features(df, target, 5)
#X = kbest_features(df, new_target, 5)
#X_clear, target_clear = remove_outliers(X, target)

score = []
inhert = []
aic=[]
purity = []
avg_cl_size = []
plt.figure(figsize=(12.0,12.0))
X, _ = lasso_features(df, target, 50)
clusters = range(2,30)
for k in clusters:
    # Declaring Model
    model = KMeans(n_clusters=k)

    # Fitting Model
    model.fit(X)
    score.append(silhouette_score(X, model.labels_))
    inhert.append(model.inertia_)
    purity.append(compute_pur(model.labels_,target))
    avg_cl_size.append(compute_size(model.labels_))
plt.plot(  clusters, score)
plt.plot(  clusters, inhert)
plt.xticks(clusters,rotation=45)
plt.xlabel("K number")
plt.ylabel("Score")
plt.title("K-means scores for mutliple number of clusters")
plt.show()
plt.clf()
plt.figure(figsize=(12.0,12.0))

plt.plot( clusters, purity)
plt.title("Cluster purity")
plt.show()
plt.figure(figsize=(12.0,12.0))


plt.plot( clusters, avg_cl_size)
plt.title("Avg cluster size")
plt.show()


In [ ]:
#---------------------------------------------------------------------------
#FLAG  -> 0: that cannot be executed in any machine, excpet Alex.
#--------------------------------------------------------------------------

#print(len(model.labels_))
#print(model.labels_[0])
#print(type(model.labels_[0]))
#print(type(target[1]))
#print(target[1])
def compute_pur(labels,target):
    d = defaultdict(lambda: [])
    for i in range(len(labels)):
        d[int(labels[i])].append(int(target[i]))
    cluster_purity = []
    for c in d:
        max_tar = max(d[c])
        cluster_purity.append(max_tar/len(d[c]))
    return sum(cluster_purity)/ len(cluster_purity)

def compute_size(labels):
    d = defaultdict(lambda: [])
    for i in range(len(labels)):
        d[int(labels[i])].append(int(target[i]))
    clusters = [len(d[x]) for x in d]
    
    return sum(clusters)/ len(clusters)

In [ ]:
#---------------------------------------------------------------------------
#FLAG  -> 0: that cannot be executed in any machine, excpet Alex.
#--------------------------------------------------------------------------


# Reading the DataFrame
#seeds_df = pd.read_csv(
#    "https://raw.githubusercontent.com/vihar/unsupervised-learning-with-python/master/seeds-less-rows.csv")

# Remove the grain species from the DataFrame, save for later
#varieties = list(seeds_df.pop('grain_variety'))

# Extract the measurements as a NumPy array
#samples = seeds_df.values

"""
Perform hierarchical clustering on samples using the
linkage() function with the method='complete' keyword argument.
Assign the result to mergings.
"""
mergings = linkage(X, method='complete')

"""
Plot a dendrogram using the dendrogram() function on mergings,
specifying the keyword arguments labels=varieties, leaf_rotation=90,
and leaf_font_size=6.
"""
dendrogram(mergings,
           labels=varieties,
           leaf_rotation=90,
           leaf_font_size=6,
           )

plt.show()

In [ ]:
#---------------------------------------------------------------------------
#FLAG  -> 0: that cannot be executed in any machine, excpet Alex.
#--------------------------------------------------------------------------

X, _ = lasso_features(df, target, 4)
for i in range(10,250,10):
    print("i:{}".format(i))
    tsne_2d = make_tsne(X, target, "full dataset", learning_rate=350, perplexity=i)

In [ ]:
#---------------------------------------------------------------------------
#FLAG  -> 0: that cannot be executed in any machine, excpet Alex.
#--------------------------------------------------------------------------

df, target, user_ids = read_data("features_large_extra.csv")
X, _ = lasso_features(df, target, 4)


In [ ]:
#---------------------------------------------------------------------------
#FLAG  -> 0: that cannot be executed in any machine, excpet Alex.
#--------------------------------------------------------------------------

perofrm_umap_clustering(X, target,  "Full", n_neighbors=25, n_components=30)

In [ ]:
#---------------------------------------------------------------------------
#FLAG  -> 0: that cannot be executed in any machine, excpet Alex.
#--------------------------------------------------------------------------


scores = []
for j in range(100):
    df, target, user_ids = read_data("features_large_extra.csv")
    X = rforest_features(df, target, 40)
    for i in range(1000):
        clf = XGBClassifier(objective="multi:softprob",
                            num_class = 2,
                            learning_rate =0.2,
                            n_estimators=100,
                            max_depth=5,
                            colsample_bytree = 0.7,
                            eval_metric="rmse",
                            use_label_encoder=False)
        X_train, X_test, y_train, y_test = train_test_split(X, target, test_size=0.3,)
        clf.fit(X_train, y_train)
        scores.append(clf.score(X_test,y_test))
    
print("Avg score:{}".format(sum(scores)/len(scores)))

